# Project 1  - Data Preprocessing

### Dataset - Candy Hierarchy 2017
https://www.scq.ubc.ca/so-much-candy-data-seriously/

### Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import json

### Define functions and variables

In [2]:
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_

unit_dict = np.load('dictionary.npy',allow_pickle='TRUE').item()

### Read in dataset as Pandas dataframe

In [3]:
df = pd.read_csv('./data/candyhierarchy2017.csv', encoding = "ISO-8859-1")

# Removing of unnecessary rows

In [ ]:
#drop rows base on their index value.
#To test this, drop index 0 first since that row do not have any record.
df = df.drop(0)

#Show and check if the row is deleted
df.head()

Begin dropping rows: upon observation, rows that have multiple missing data starts to miss input in the column  "Q6 | Any full-sized candy bar," so this column is used as a "filter" to drop these rows.

In [ ]:
df = df.dropna(subset = ['Q6 | Any full-sized candy bar'])
df.head()

#create a new updated candyfile dataset
df.to_csv('updated_candy_set_rows.csv')

# Removing of Unnecessary Columns
---
This is the removing of columns that are deemed as not needed for our finalized data, so as part of the preprocessing process we are removing said data.

After storing the data to a file, by looking at the survey that was provided for the candy hierarchy, columns will be dropped because they do not serve a value.

---

The following colums were removed for these reasons:
- > Internal ID: No value with our data
- > Q1 : Not a candy question
- > Q6 : Not a candy
- > Q7, Q8, Q9: Comments serving no value
- > Q10, Q11, Q13: Not a candy question
- > Unamed: 113: Blank column with no value
- > Click Coordinates (x, y): Not of significant value to candy data

In [ ]:
# All columns from the data set that will be removed
col_remove = ["Internal ID", "Q1: GOING OUT?", 
"Q6 | Anonymous brown globs that come in black and orange wrappers	(a.k.a. Mary Janes)", 
"Q6 | Any full-sized candy bar", "Q6 | Bonkers (the board game)", "Q6 | Box'o'Raisins",
"Q6 | Broken glow stick", 
"Q6 | Candy that is clearly just the stuff given out for free at restaurants", 
"Q6 | Cash, or other forms of legal tender", "Q6 | Chardonnay", 
"Q6 | Chick-o-Sticks (we donÕt know what that is)",
"Q6 | Creepy Religious comics/Chick Tracts", "Q6 | Dental paraphenalia", 
"Q6 | Generic Brand Acetaminophen", "Q6 | Glow sticks", "Q6 | Green Party M&M's", 
"Q6 | Gum from baseball cards", "Q6 | Independent M&M's",
"Q6 | Gummy Bears straight up", "Q6 | Healthy Fruit", "Q6 | Hugs (actual physical hugs)",
"Q6 | Jolly Rancher (bad flavor)", "Q6 | JoyJoy (Mit Iodine!)", "Q6 | Senior Mints",
"Q6 | Kale smoothie", "Q6 | Abstained from M&M'ing.", "Q6 | Pencils", 
"Q6 | Mint Juleps", "Q6 | Spotted Dick", "Q6 | Minibags of chips", 
"Q6 | Real Housewives of Orange County Season 9 Blue-Ray",
"Q6 | Sandwich-sized bags filled with BooBerry Crunch",
"Q6 | Those odd marshmallow circus peanut things", 
"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein", 
"Q6 | Vicodin", "Q6 | White Bread", "Q6 | Whole Wheat anything", 
"Q7: JOY OTHER", "Q8: DESPAIR OTHER", "Q9: OTHER COMMENTS", "Q10: DRESS", 
"Unnamed: 113", "Q11: DAY", "Q12: MEDIA [Daily Dish]", "Q12: MEDIA [Science]", "Q12: MEDIA [ESPN]", 
"Q12: MEDIA [Yahoo]", "Click Coordinates (x, y)"]

# Files are dropped from the dataframe
df = df.drop(columns = col_remove)

# New dataframe is saved to relative file location and name
df.to_csv('candydata_col_removed.csv')

### Columns in the data are now removed and stored to the variable in the final line of code above.

In [5]:
df = df.dropna(subset = ['Q6 | Any full-sized candy bar'])
df.head()

#create a new updated candyfile dataset
df.to_csv('updated_candy_set_rows.csv')

## After this process, rows are now sorted with the ones that have multiple data entry

# Cleaning Age Values

Removing rows where age is not an integer

In [ ]:
df = df[pd.to_numeric(df['Q3: AGE'], errors='coerce').notnull()]

# Normalizing Genders
 Normalizing and mapping to values

Replacing NaN with "I'd rather not say" since it makes more sense, rather than just deleting...

In [ ]:
df['Q2: GENDER'].fillna(value = "I'd rather not say", inplace = True)

Maps a value to each...

In [ ]:
df2 = {"Male": '0', 'Female': '1', "I'd rather not say" : '2', 'other' : '3' }
df = df.replace({"Q2: GENDER": df2})

# Normalizing Countries

# Normalizing States, Cities, Provinces

Convert dictionary and target column to uppercase before mapping

In [ ]:
unit_dict =  {k.upper(): v for k, v in unit_dict.items()}
df['Q5: STATE, PROVINCE, COUNTY, ETC'] = df['Q5: STATE, PROVINCE, COUNTY, ETC'].str.upper()

Map values in column to their corresponding dictionary key

In [ ]:
df['Q5: STATE, PROVINCE, COUNTY, ETC'] = df['Q5: STATE, PROVINCE, COUNTY, ETC'].map(unit_dict).fillna(df['Q5: STATE, PROVINCE, COUNTY, ETC'])

Display counts of unique values after mapping

In [ ]:
df['Q5: STATE, PROVINCE, COUNTY, ETC'].value_counts()

Label encode each state or province after mapping

In [ ]:
encode_text_index(df, 'Q5: STATE, PROVINCE, COUNTY, ETC')

# Normalizing Candy Ratings

In [ ]:
for index in df.columns:
    df[index] = df[index].replace(['MEH', 'JOY', 'DESPAIR'], ['1', '2', '0'])

print(df)

## Replacing missing values options
---
### Replace with meh
    > file[index] = file[index].fillna(1)
- Decided upon this for candy sections because mean would not work for our categorical needs

### Replace with new value
    > file[index] = file[index].fillna(2)
- Decided upon this for gender and location

### Replace with mean
    > To use/do mean, make sure that data is in int
    > file[index] = file[index].fillna(file[index].mean())
- Decided upon this for age

In [ ]:
# This for loop loops through every column in the data set
for index in df.columns:
    if index == 'Q2: GENDER':
        # Gender fill in | Creating a third option of other/nan
        df[index] = df[index].fillna(2)
    elif index == 'Q3: AGE':
        # Age fill in | Age must be normalized for this to work (no strings)
        df[index] = df[index].fillna(df[index].mean())
        pass
    elif index == 'Q5: STATE, PROVINCE, COUNTY, ETC':
        # Country & State/Province fill in | Creating a third option of other/nan
        df[index] = df[index].fillna(2)  
    else: # If it is in the candy column
        df[index] = df[index].fillna(1)

df.head()